In [ ]:
#Train, val, Test splitter
import train_pytorch
import dataset_utils as utils
import random
import time 
from collections import Counter
import os
import numpy as np
import shutil
import codecs

In [ ]:
language = "EN"
raw_data_dir = "raw_data/derivatives/"
annotation_file = f"raw_data/annotation/lpp{language}_word_information.csv"
random.seed(1234)

pos = "ADJ" #Change this to select labels


In [ ]:
#Run this cell to load the data with the selected configuration, only need to run this cell if 
#you haven't loaded the data before, or if you want to change config

#Uncomment these two if you are changing config
# utils.clear_data_dir()
# shutil.rmtree("aux_data/")

utils.raw2npy(raw_data_dir, language=language)
config_version = "inter_subject" # // "inter_subject"
data_dir = "data"
split = (0.33, 0.33, 0.33) #train, val, test
pos = "VERB" #Part-Of-Speech
utils.prepare_dataset(config_version, 
                        data_dir, 
                        annotation_file, 
                        split, 
                        language, 
                        pos)



In [ ]:
#Finds the labels 
all_labels = []
for i in ["Train", "Test", "Val"]:
    if os.path.isdir(i):
        for run in range(9):
            path = os.path.join(i, f"data/{run}/{language}/labels.txt")
            with open(path, "r") as f:
                lines = f.readlines()
                for l in lines:
                    all_labels.append(l.strip("\n"))


import json
most_common = Counter(all_labels).most_common()
with open("labels.json", "r", encoding="utf-8") as f:
    label_dict = json.load(f)

label_dict = {v:k for k,v in label_dict.items()}

most_common_lemmas = []
for i in range(len(most_common)):
    lbl = int(most_common[i][0])
    most_common_lemmas.append((label_dict[lbl], most_common[i][1]))


In [ ]:
top = 15
vocab = []
off_set = 0 #how many to exclude from the most frequent words to stop overfitting
label_stats = most_common_lemmas[off_set+1:top+off_set+1]
for w, _ in label_stats: #top 15 words
    vocab.append(w.strip())
    
print(vocab)
print(label_stats)

In [ ]:
curated_labels = utils.prepare_handpicked_labels(annotation_file, vocab)
utils.create_label_files(curated_labels, language)

In [ ]:
batch_size=50
#config = f"{config_version}_{language}_top15{pos}"
config = f"config2_{language}_top15{pos}"

In [ ]:
with open("labels.json", "r") as f:
    oov = json.load(f)["oov"]

train_pytorch.train(15, batch_size = batch_size, num_epochs = 10, language=language, config=config, model = "2d", weights=None, oov=oov)

In [ ]:
import model_m2dcnn as model
from torch.utils.data import DataLoader
from dataset import lpp_Dataset
batch_size = 50
binary = False
test_dataloader = DataLoader(lpp_Dataset("Test/data", language=language, ignoreOOV=oov), batch_size=batch_size, shuffle=False)
if binary:
        nb_classes = 2
else:
    with open("labels.json", "r", encoding="utf-8") as f:
        nb_classes = len(json.load(f).keys())
      
print(nb_classes)
train_pytorch.test(model.M2DCNN(numClass=nb_classes, 
                                numFeatues=30880, 
                                DIMX=74, DIMY=90, DIMZ=73), 
                                config,
                                f"results/{config}_weights.pth", 
                                nb_classes, 
                                test_dataloader)